In [1]:
def evaluate_association_rules(association_rules,test_datas):
    """
    Parameters：
        association_rules：要评估的关联规则（推荐规则） list((rule_a,rule_b))
        test_datas：测试用的数据集 dataframe

    Returns：
        关联规则（推荐规则）在测试集上的平均准确率
    """
    point_sum = 0
    miss = 0
    for rule_a,rule_b in association_rules:
        set_rule_a = set(rule_a)
        set_rule_b = set(rule_b)
        
        num_a = 0
        num_b = 0
        
        for test_data in test_datas.value:
            set_test_data = set(test_data)
            if set_rule_a.issubset(set_test_data):
                num_a += 1
                if set_rule_b.issubset(set_test_data):
                    num_b += 1
        if num_a:
            point_sum += num_b / num_a
        else:
            miss += 1
    print(miss,' rules miss')
    return point_sum / len(association_rules)
    

### 函数测试

In [2]:
# 导入所需模块
import pandas as pd
import numpy as np
import json
import fp_growth_py3 as fpg
import matplotlib.pyplot as plt
import pylab as pl
from collections import defaultdict
import association_rules 

10339
<class 'generator'>
-------------挖掘频繁项集---------------
                fluent_patterns   support
0                      [102392]  0.315795
1                        [8892]  0.269272
2                [102392, 8892]  0.142083
3                [139252, 8892]  0.141213
4        [102392, 139252, 8892]  0.086952
5               [5267730, 8892]  0.144405
6       [102392, 5267730, 8892]  0.086179
7       [139252, 5267730, 8892]  0.103008
8                     [4312482]  0.205726
9             [102392, 4312482]  0.099139
10              [8892, 4312482]  0.103105
11      [139252, 8892, 4312482]  0.075249
12     [5267730, 8892, 4312482]  0.075152
13            [139252, 4312482]  0.123803
14           [5267730, 4312482]  0.120708
15   [139252, 5267730, 4312482]  0.091208
16            [130412, 4312482]  0.097011
17    [139252, 130412, 4312482]  0.076700
18   [5267730, 130412, 4312482]  0.072831
19                     [135652]  0.194893
20             [102392, 135652]  0.088306
21             

In [3]:
# 读入测试数据集
filepath='./test_data/user_following_animation.json'
data=pd.read_json(filepath,lines=True)
user_info = pd.read_csv("test_data/bilibili_crawler_user_info.csv",names = ['id','mid','name','sex','sign','the_rank','level','jointime','moral','silence','birthday','coins','fans_badge','role','title','desc','vip_type','vip_status'])

len(user_info)


51432

In [4]:
user_info.drop(user_info[user_info.vip_type.isna() | user_info.the_rank.isna() | user_info.level.isna()].index.tolist(),inplace=True) # 将vip_type、the_rank、level有NaN的行去掉
len(user_info)

51424

In [5]:
# 设置权重字典
add_rules={'the_rank':{10000:1,20000:2,25000:3,30000:4},'level':{'3':1,'4':2,'5':3,'6':4},'vip_type':{0:0,1:1,2:2}}


In [6]:
# 去除一下异常数据
for index,row in data.iterrows():
    try:
        int(row.key)
    except:
        print(row)
        data.drop(index,inplace=True)

db                                                       7
key                                         finished_users
size                                                507576
ttl                                                     -1
type                                                   set
value    [330817737, 74775, 259640193, 24774761, 540994...
Name: 4504, dtype: object


In [7]:
def user_power(data,rules,user_info):
    """
    params:
        data:用户收藏ID数据集
        rules:权重规则
        user_info:用户信息数据集
    return:新数据集
        
    """
    new_data = pd.DataFrame(columns=data.columns)
    for index,row in data.iterrows():
        info = user_info[user_info.mid == int(row.key)]

        if len(info):
            the_power = rules['the_rank'][info.the_rank.values[0]] + rules['level'][info.level.values[0]] + rules['vip_type'][info.vip_type.values[0]]
        else:  # len(info)==0 说明在user_info中没有这个用户的相关数据
            the_power = 1
#         print(the_power)
        for i in range(the_power):
            new_data = new_data.append(row,ignore_index=True)
    return new_data

    

In [8]:
# 根据权重规则对数据进行扩充
new_data= user_power(data,add_rules,user_info)
len(new_data)

38829

将测试数据集 划分为training_data与evaluate_data两部分，占比可调

In [9]:
training_data = new_data.sample(n=None, frac=0.9, replace=False, weights=None, random_state=None, axis=None)

In [10]:
print(len(training_data))
training_data.head()

34946


,db,key,size,ttl,type,value
34168,7,21504568,184,-1,set,"[184, 835, 963, 964, 1624, 2576, 3023, 3287, 3..."
29685,7,5024346,81,-1,set,"[2722, 2726, 2742, 2743, 3464, 5056, 5057, 125..."
19540,7,1358899,357,-1,set,"[190, 333, 530, 1389, 1446, 1535, 1537, 1590, ..."
33213,7,6521637,168,-1,set,"[184, 679, 1089, 1192, 1552, 1553, 1635, 1757,..."
26058,7,27249897,29,-1,set,"[5851, 5978, 6159, 11712, 78052, 4312762]"


In [11]:
evaluate_data = new_data.drop([x for x in training_data.index])

In [12]:
print(len(evaluate_data))
evaluate_data.head()

3883


,db,key,size,ttl,type,value
21,7,5667082,526,-1,set,"[282, 289, 290, 311, 572, 687, 713, 723, 735, ..."
29,7,344488725,4,-1,set,[5744]
34,7,36562891,20,-1,set,"[1586, 2543, 3287, 3461, 5997]"
43,7,34709356,263,-1,set,"[184, 685, 709, 710, 856, 1588, 1665, 1672, 17..."
52,7,2598372,47,-1,set,"[1559, 1699, 1733, 5559, 5626, 5852, 6402, 875..."


用training_data进行训练，导出关联规则（推荐规则）

注：这里只是用置信度进行了排名，因为只是对评估函数进行测试，所以没有加入更多的排名方法。

In [65]:

data_list = list(training_data["value"])
frequent_itemsets = fpg.find_frequent_itemsets(data_list, minimum_support=0.07 * len(data_list), include_support=True)
print(type(frequent_itemsets))  # print type
result = []
for itemset, support in frequent_itemsets:  # 将generator结果存入list
    result.append((itemset, support / len(data_list)))

result_patterns = [i[0] for i in result]
result_support = [i[1] for i in result]
patterns_df = pd.DataFrame({"fluent_patterns": result_patterns, "support": result_support})
patterns = {}
for i in result:
    patterns[frozenset(sorted(i[0]))] = i[1]
print("-------------挖掘频繁项集---------------")
print(patterns_df)

def generate_rules(patterns, min_confidence):
    patterns_group = group_patterns_by_length(patterns)
    raw_rules = defaultdict(set)
    for length, pattern_list in patterns_group.items():
        if length == 1:
            continue
        for pattern, support in pattern_list:
            item_list = list(pattern)
            for window_size in range(1, length):
                for i in range(0, length - window_size):
                    for j in range(i + window_size, length):
                        base_set = frozenset(item_list[i:j])
                        predict_set = frozenset(pattern - base_set)
                        confidence = support / patterns.get(base_set)
                        if confidence > min_confidence:
                            raw_rules[base_set].add((predict_set, confidence))

                        base_set, predict_set = predict_set, base_set
                        confidence = support / patterns.get(base_set)
                        if confidence > min_confidence:
                            raw_rules[base_set].add((predict_set, confidence))
    return raw_rules

def group_patterns_by_length(patterns):
    result = defaultdict(list)
    for pattern, support in patterns.items():
        result[len(pattern)].append((pattern, support))
    return result

def transform(raw_rules):
    result = list()
    for base_set, predict_set_list in raw_rules.items():
        for predict_set, confidence in predict_set_list:
            result.append((base_set, predict_set, confidence))

    return result
raw_rules = generate_rules(patterns, 0.6)
rules = transform(raw_rules)
rules.sort(key=lambda x: x[2], reverse=True)
rules_a = [i[0] for i in rules]
rules_b = [i[1] for i in rules]
confidence = [i[2] for i in rules]
rules_df = pd.DataFrame({"rules_a": rules_a, "rules_b": rules_b, "confidence": confidence})
print(rules_df)

<class 'generator'>
-------------挖掘频繁项集---------------
             fluent_patterns   support
0                     [8892]  0.305099
1             [139252, 8892]  0.166772
2     [102392, 139252, 8892]  0.101528
3             [102392, 8892]  0.160362
4                     [3461]  0.216849
5               [8892, 3461]  0.112660
6       [139252, 8892, 3461]  0.070366
7       [102392, 8892, 3461]  0.072397
8             [139252, 3461]  0.108138
9            [5267730, 3461]  0.104046
10   [139252, 5267730, 3461]  0.073513
11     [8892, 5267730, 3461]  0.070995
12            [102392, 3461]  0.112087
13              [5997, 3461]  0.112688
14      [102392, 5997, 3461]  0.070480
15        [8892, 5997, 3461]  0.072283
16            [130412, 3461]  0.091370
17                    [5852]  0.208178
18              [8892, 5852]  0.097522
19              [3461, 5852]  0.075116
20            [102392, 5852]  0.102530
21            [139252, 5852]  0.090683
22              [5997, 5852]  0.074486
23       

In [66]:
import association_rules
import imp
imp.reload(association_rules)
# 读取animation 和 animation_feature
animation = pd.read_json("./test_data/bilibili_crawler_animation.json", encoding="utf-8")
animation["score"] = animation["score"].fillna('%.1f' % animation["score"].mean())
animation[["follow", "play"]] = animation[["follow", "play"]].applymap(association_rules.trans)
animation_feature = pd.read_json("./test_data/bilibili_crawler_animation_feature.json",dtype={"character_voice_list": str})
animation_feature[["tag_list", "character_voice_list", "character_staff_list"]] = animation_feature[["tag_list", "character_voice_list", "character_staff_list"]].applymap(json.loads)

# #将关联规则的后项展开
rules_df = association_rules.unfold_rules(rules_df)
# 对关联规则进行扩展，综合考虑置信度、番剧评分、声优导演等信息
rules_weight = {"confidence": 0.5, "score": 0.1, "play": 0.1, "follow": 0.1, "voice": 0.1, "staff": 0.1}
rules_df = association_rules.add_score(rules_df, rules_weight, animation, animation_feature)
for index,row in rules_df.iterrows():
    print(row["rules_a"], row["rules_b"], row["confidence"], row["score"])

10339
<class 'generator'>
-------------挖掘频繁项集---------------
                fluent_patterns   support
0                      [102392]  0.315795
1                        [8892]  0.269272
2                [102392, 8892]  0.142083
3                [139252, 8892]  0.141213
4        [102392, 139252, 8892]  0.086952
5               [5267730, 8892]  0.144405
6       [102392, 5267730, 8892]  0.086179
7       [139252, 5267730, 8892]  0.103008
8                     [4312482]  0.205726
9             [102392, 4312482]  0.099139
10              [8892, 4312482]  0.103105
11      [139252, 8892, 4312482]  0.075249
12     [5267730, 8892, 4312482]  0.075152
13            [139252, 4312482]  0.123803
14           [5267730, 4312482]  0.120708
15   [139252, 5267730, 4312482]  0.091208
16            [130412, 4312482]  0.097011
17    [139252, 130412, 4312482]  0.076700
18   [5267730, 130412, 4312482]  0.072831
19                     [135652]  0.194893
20             [102392, 135652]  0.088306
21             

{11712, 5267730} 8892 0.674724119810825 1.413500326346989
{134932, 4316382} 8892 0.6742404901710493 1.413258511527101
{130412} 139252 0.673541076487252 1.4537350865200516
{4316482, 22718131} 130412 0.6733369388858842 1.306282366954242
{5267730, 102252} 8892 0.6728314665277417 1.4125539997054473
{5267730, 102252} 4316382 0.6728314665277417 1.2893731932831125
{139252, 4316382} 8892 0.6726892354507418 1.4124828841669472
{4312482, 130412} 8892 0.6725777435106657 1.4124271381969091
{4316482, 130412} 4316382 0.672053872053872 1.2889843960461778
{102252} 8792 0.6717926811903745 1.4512948775081826
{102392, 4312482} 5267730 0.67173017507724 1.436785779509794
{102392, 4316382} 5267730 0.6712902381575504 1.4365658110499493
{102392, 4316382} 139252 0.6712902381575504 1.4526096673552007
{130412, 8892, 139252} 4316482 0.6709857723577236 1.1788668809872629
{102792} 102392 0.6706904876871077 1.4831230216213316
{102392, 130412} 5267730 0.6695754716981133 1.4357084278202308
{8792, 139252} 8892 0.6687555

In [ ]:
# 将规则转换为(rule_a,rule_b)的格式，根据rules的实际情况更改
new_rule = []
for a,b,c in rules:
    new_rule.append([a,b])

In [ ]:
new_rule[:5]

In [62]:
# 计算前100条规则的平均准确率
result = evaluate_association_rules(new_rule[:100],evaluate_data)
result

0  rules miss


0.8622445554900482

In [63]:
# 计算第100到200条规则的平均准确率
result = evaluate_association_rules(new_rule[100:200],evaluate_data)
result

0  rules miss


0.7885238020494877

In [64]:
# 计算第200到300条规则的平均准确率
result = evaluate_association_rules(new_rule[200:300],evaluate_data)
result

0  rules miss


0.738594195743888

In [65]:
# 计算全部规则的平均准确率
result = evaluate_association_rules(new_rule,evaluate_data)
result

0  rules miss


0.7181457165689142

分别对前100条规则、第100到200条规则、第200到300条规则、全部规则进行评估，
发现使用靠近前面的规则进行推荐，平均准确率较高，证明了使用关联规则的置信度对规则进行排名有一定的优化效果。
